In [28]:
include("src\\building_tree.jl")
include("src\\utilities.jl")
include("src\\merge.jl")
using Debugger
using DataFrames, Random
using DataStructures
using StatsBase
using Statistics
using LinearAlgebra
using LazySets
using Polyhedra

In [29]:
dataSetName = "iris"
include("./data/" * dataSetName * ".txt")
# Ramener chaque caractéristique sur [0, 1]
reducedX = Matrix{Float64}(X)
for j in 1:size(X, 2)
    reducedX[:, j] .-= minimum(X[:, j])
    reducedX[:, j] ./= maximum(X[:, j])
end

train, test = train_test_indexes(length(Y))
X_train = reducedX[train,:]
Y_train = Y[train]
X_test = reducedX[test,:]
Y_test = Y[test]
classes = unique(Y)

println(" (train size ", size(X_train, 1), ", test size ", size(X_test, 1), ", ", size(X_train, 2), ", features count: ", size(X_train, 2), ")")

# Temps limite de la
time_limit = 10
D = 3
x = X_train
y = Y_train

 (train size 120, test size 30, 4, features count: 4)


120-element Vector{Int64}:
 3
 1
 3
 1
 1
 1
 1
 1
 2
 1
 ⋮
 3
 1
 2
 3
 1
 1
 1
 2
 1

In [30]:
n = length(Y_train)
m = length(x[1,:])
clusters = Vector{Cluster}([])
for dataId in 1:size(x,1)
    push!(clusters, Cluster(dataId, x,y))
end
clusterId = collect(1:n) #On obtient un vecteur 1,2..., qui correspond pour chaque cluster à son clusterId
distances = Vector{Distance}([])
for id1 in 1:n-1
    for id2 in id1+1:n
        push!(distances, Distance(id1, id2, x))
    end
end
sort!(distances, by = v ->v.distance)
remainingClusters=n
distanceId = 1
n_epochs = 1
c1_bis = Nothing
c2_bis = Nothing
i = 1
a=i
max_elements_small_classes = 5
num_clusters = 10

while remainingClusters>= num_clusters
    distance = distances[distanceId]
    cId1 =clusterId[distance.ids[1]]
    cId2 = clusterId[distance.ids[2]]
    if cId1 != cId2
        c1 = clusters[cId1]
        c2 = clusters[cId2]
        count_classes_in_clusters = StatsBase.countmap([y[c2.dataIds]; y[c1.dataIds]])
        df = DataFrame()
        df."classes" = collect(keys(count_classes_in_clusters))
        df."count_classes" = collect(values(count_classes_in_clusters))
        df = sort(df, [:count_classes], rev=true)
        # if sum(df[df."count_classes".!=maximum(df."count_classes"),:]."count_classes") <= max_elements_small_classes
        if sum(df[df."classes".!=df[1,1],"count_classes"])<= max_elements_small_classes
            remainingClusters -=1
            if remainingClusters < num_clusters
                break
            end
            merge!(c1, c2) #On merge les 2 clusters
            for id in c2.dataIds 
                clusterId[id]= cId1 #On modifie le clusterId dans la serie pour le cluster_2, on lui affecte le cluster_1
            end
            # Vider le second cluster
            empty!(clusters[cId2].dataIds)
        end

    end
    distanceId += 1
    # if i % 50 == 0
    # #     println(remainingClusters)
    # #     # print("\n")
    # # end
end
df_clusters = DataFrame()
df_clusters."cluster_id" = collect(keys(StatsBase.countmap(clusterId)))
df_clusters."number_elements" = collect(values(StatsBase.countmap(clusterId)))
# higher_than_threshold(value::Int64) = value >= 10
# clusters_to_treat = filter(:"number_elements"=> higher_than_threshold, df_clusters)."cluster_id"



10-element Vector{Int64}:
  7
  1
  1
 21
 43
 40
  1
  1
  1
  4

In [33]:
for cluster_interest in clusters_to_treat 
    count_classes_in_clusters = StatsBase.countmap(y[findall(x->x==cluster_interest, clusterId)])
    y_cluster = y[findall(x->x==cluster_interest, clusterId)]
    df = DataFrame()
    df."classes" = collect(keys(count_classes_in_clusters))
    df."count_classes" = collect(values(count_classes_in_clusters))
    df = sort(df, [:count_classes], rev=true)
    main_class = 0
    if size(df)[1]>1
        main_class = df[1,1]
        print("several classes in cluster")
    else
        println("only one class in cluster")
        println(", we are not touching cluster ", cluster_interest)
        main_class = df[1,1]
    end   
    points_in_cluster = findall(x->x==cluster_interest, clusterId)
    points_reckon_with = Vector{Int64}([])
    points_sideline = Vector{Int64}([])
    for (i, point) in enumerate(points_in_cluster)
        if y[point] != main_class
            push!(points_sideline,i )
        else
            push!(points_reckon_with, i)
        end
    end
    matrix_interest = clusters[cluster_interest].x
    size(matrix_interest, 1)
    vectors_cluster = [matrix_interest[i,:] for i in 1:size(matrix_interest,1)]
    vectors_cluster[points_reckon_with]
    hull = LazySets.convex_hull(vectors_cluster[points_reckon_with])
    for i in 1:length(points_sideline)
        println(i, " ",element(Singleton(vectors_cluster[points_sideline[i]])) ∈ VPolytope(hull))
        if element(Singleton(vectors_cluster[points_sideline[i]])) ∈ VPolytope(hull)
            push!(points_reckon_with,  points_sideline[i])
        end
    end
    sort!(points_reckon_with)
    points_to_be_taken_out_cluster = points_sideline[points_sideline .∉ Ref(points_reckon_with)]

    new_clusters = Vector{Cluster}([])
    point_cluster  = points_reckon_with[1]
    trimmed_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
    for point_cluster in points_reckon_with[2:end]
        new_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
        merge!(trimmed_cluster, new_cluster)
    end

    for point_cluster in points_to_be_taken_out_cluster
        new_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
        push!(clusters, new_cluster)
    end
    clusters[cluster_interest] = trimmed_cluster
    print(length(clusters))
end

only one class in cluster
, we are not touching cluster 41


glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to r

several classes in clusterglp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution